EXPERIMENT 1

In [ ]:
import collections
from collections import deque
import time

class PuzzleNode:
    def __init__(self, state, parent=None, action=None, depth=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.depth = depth
        self.state_list = list(state)
        self.blank_pos = self.state_list.index(0)

    def __hash__(self):

        return hash(self.state)

    def __eq__(self, other):

        if isinstance(other, PuzzleNode):
            return self.state == other.state

        return self.state == other

    def get_successors(self):

        successors = []

        r, c = divmod(self.blank_pos, 3)


        moves = [(-1, 0, 'UP'), (1, 0, 'DOWN'), (0, -1, 'LEFT'), (0, 1, 'RIGHT')]

        for dr, dc, action in moves:
            nr, nc = r + dr, c + dc


            if 0 <= nr < 3 and 0 <= nc < 3:

                new_blank_pos = nr * 3 + nc

                new_state_list = self.state_list[:]


                new_state_list[self.blank_pos], new_state_list[new_blank_pos] = \
                    new_state_list[new_blank_pos], new_state_list[self.blank_pos]

                new_state_tuple = tuple(new_state_list)

                successors.append(PuzzleNode(
                    new_state_tuple,
                    parent=self,
                    action=action,
                    depth=self.depth + 1
                ))

        return successors


def reconstruct_path(node):
    """Traces the path from the goal node back to the root. """
    path = []
    current = node
    while current:
        path.append((current.state, current.action))
        current = current.parent
    return list(reversed(path))

def pretty_print_state(state):
    """Prints the 3x3 grid state."""
    output = []
    for i in range(0, 9, 3):

        row = [str(state[j]) if state[j] != 0 else ' ' for j in range(i, i + 3)]
        output.append(f" {' '.join(row)}")
    return '\n'.join(output)

def solve_puzzle(initial_state, goal_state, strategy='bfs', limit=30):

    root = PuzzleNode(initial_state)
    goal = PuzzleNode(goal_state)

    if root.state == goal.state:
        return [(initial_state, None)], 0


    frontier = deque([root]) if strategy == 'bfs' else [root]

    visited = {root.state}
    nodes_explored = 0

    while frontier:
        nodes_explored += 1

        if strategy == 'bfs':
            current_node = frontier.popleft()
        else:
            current_node = frontier.pop()

        if current_node.state == goal.state:

            return reconstruct_path(current_node), nodes_explored

        if strategy == 'dls' and current_node.depth >= limit:
            continue

        for successor in current_node.get_successors():
            if successor.state not in visited:
                visited.add(successor.state)
                frontier.append(successor)

    return None, nodes_explored

if __name__ == "__main__":

    INITIAL_STATE = (1, 2, 3, 4, 5, 6, 0, 7, 8)
    GOAL_STATE = (1, 2, 3, 4, 5, 6, 7, 8, 0)


    start_time_bfs = time.time()
    path_bfs, explored_bfs = solve_puzzle(INITIAL_STATE, GOAL_STATE, 'bfs')
    time_bfs = time.time() - start_time_bfs

    DLS_LIMIT = 10
    start_time_dls = time.time()
    path_dls, explored_dls = solve_puzzle(INITIAL_STATE, GOAL_STATE, 'dls', limit=DLS_LIMIT)
    time_dls = time.time() - start_time_dls


    def format_output(path, explored, time_taken, strategy_name, limit=None):
        moves = len(path) - 1 if path else 'N/A'
        limit_str = f" (Limit={limit})" if limit else ""

        return (
            f"\nAttempting {strategy_name}{limit_str}...\n"
            f"Solution found: {'YES' if path else 'NO'}\n"
            f"Solution found in {moves} moves.\n"
            f"Nodes explored: {explored}\n"
            f"Time taken: {time_taken:.4f} seconds"
        )

    print("=" * 30)
    print("       8-PUZZLE SOLVER       ")
    print("=" * 30)
    print("Initial State:")
    print(pretty_print_state(INITIAL_STATE))
    print("\nGoal State:")
    print(pretty_print_state(GOAL_STATE))

    print(format_output(path_bfs, explored_bfs, time_bfs, "Breadth-First Search (BFS)"))
    print(format_output(path_dls, explored_dls, time_dls, "Depth-Limited Search (DLS)", DLS_LIMIT))

       8-PUZZLE SOLVER       
Initial State:
 1 2 3
 4 5 6
   7 8

Goal State:
 1 2 3
 4 5 6
 7 8  

Attempting Breadth-First Search (BFS)...
Solution found: YES
Solution found in 2 moves.
Nodes explored: 7
Time taken: 0.0001 seconds

Attempting Depth-Limited Search (DLS) (Limit=10)...
Solution found: YES
Solution found in 2 moves.
Nodes explored: 3
Time taken: 0.0000 seconds


EXPERIMENT 2

In [ ]:

N = 8

def is_safe(board, row, col):

    for prev_col in range(col):

        if board[prev_col] == row:
            return False


        if abs(board[prev_col] - row) == abs(prev_col - col):
            return False

    return True


def solve_8_queens_util(board, col, N, solutions):



    if col >= N:
        solutions.append(board[:])
        return True


    for row in range(N):
        if is_safe(board, row, col):
            board[col] = row

            if solve_8_queens_util(board, col + 1, N, solutions):
                return True



    return False

def solve_8_queens(N=8):

    board = [-1] * N
    solutions = []


    solve_8_queens_util(board, 0, N, solutions)

    if not solutions:
        return None

    return solutions[0]


def display_board(solution):

    if not solution:
        return

    N_board = len(solution)
    output = []

    for row in range(N_board):
        line = []
        for col in range(N_board):
            if solution[col] == row:
                line.append(' Q ')
            else:
                line.append(' . ')
        output.append("".join(line))

    return "\n".join(output)

if __name__ == "__main__":

    solution = solve_8_queens(N)

    print("\n" + "=" * 50)
    print("      8-QUEENS PROBLEM SOLVER (Backtracking)     ")
    print("=" * 50)

    if solution:
        print("\nSolution Found (Row position for each column, index 0 to 7):")
        print(solution)

        print("\nBoard Visualization (Q = Queen, . = Empty):")
        print(display_board(solution))
    else:
        print("\nNo solution exists for N=8.")


      8-QUEENS PROBLEM SOLVER (Backtracking)     

Solution Found (Row position for each column, index 0 to 7):
[0, 4, 7, 5, 2, 6, 1, 3]

Board Visualization (Q = Queen, . = Empty):
 Q  .  .  .  .  .  .  . 
 .  .  .  .  .  .  Q  . 
 .  .  .  .  Q  .  .  . 
 .  .  .  .  .  .  .  Q 
 .  Q  .  .  .  .  .  . 
 .  .  .  Q  .  .  .  . 
 .  .  .  .  .  Q  .  . 
 .  .  Q  .  .  .  .  . 


EXPERIMENT 3

In [ ]:
import time

def solve_cryptarithmetic(puzzle):


    words = [w.strip() for w in puzzle.split() if w.isalpha()]

    if len(words) < 2:
        return

    unique_letters = set("".join(words))
    leading_letters = set(word[0] for word in words)

    if len(unique_letters) > 10:
        return "Error: More than 10 unique letters."

    letters_list = sorted(list(unique_letters))

    def is_solution_valid(mapping):

        def word_to_num(word):
            num = 0
            for char in word:
                num = num * 10 + mapping[char]
            return num

        for word in words:
            if len(word) > 1 and mapping[word[0]] == 0:
                return False

        addends = [word_to_num(w) for w in words[:-1]]
        result = word_to_num(words[-1])

        return sum(addends) == result

    def backtrack(index, current_mapping, used_digits):

        if index == len(letters_list):
            if is_solution_valid(current_mapping):
                return current_mapping
            return None

        letter = letters_list[index]

        for digit in range(10):
            if digit not in used_digits:

                if letter in leading_letters and digit == 0:
                    continue

                current_mapping[letter] = digit
                used_digits.add(digit)

                result = backtrack(index + 1, current_mapping, used_digits)

                if result:
                    return result

                del current_mapping[letter]
                used_digits.remove(digit)

        return None

    initial_mapping = {}
    initial_used_digits = set()
    return backtrack(0, initial_mapping, initial_used_digits)

if __name__ == "__main__":

    puzzle = "SEND + MORE = MONEY"
    start_time = time.time()
    solution = solve_cryptarithmetic(puzzle)
    end_time = time.time()

    print("\n" + "=" * 50)
    print("      CRYPTARITHMETIC PROBLEM SOLVER (Backtracking)     ")
    print("=" * 50)
    print(f"Puzzle: {puzzle}")
    print(f"Time taken: {end_time - start_time:.4f} seconds\n")

    if solution and isinstance(solution, dict):
        print("\nSolution Found:")
        print(f"Letters: {solution}")

        mapping = solution

        S, E, N, D = mapping['S'], mapping['E'], mapping['N'], mapping['D']
        M, O, R, Y = mapping['M'], mapping['O'], mapping['R'], mapping['Y']

        send = S * 1000 + E * 100 + N * 10 + D
        more = M * 1000 + O * 100 + R * 10 + E
        money = M * 10000 + O * 1000 + N * 100 + E * 10 + Y # Corrected length/power for MONEY

        print("\nVerification:")
        print(f" {send:4}")
        print(f"+{more:4}")
        print(f"-----")
        print(f"{money:5}")

    elif isinstance(solution, str) and "Error" in solution:
        print(solution)
    else:
        print("\nNo unique solution found.")


      CRYPTARITHMETIC PROBLEM SOLVER (Backtracking)     
Puzzle: SEND + MORE = MONEY
Time taken: 3.0680 seconds


Solution Found:
Letters: {'D': 7, 'E': 5, 'M': 1, 'N': 6, 'O': 0, 'R': 8, 'S': 9, 'Y': 2}

Verification:
 9567
+1085
-----
10652


EXPERIMENT 4

In [ ]:
import heapq

def a_star_search(graph, heuristic, start, goal):

    open_set = []


    all_nodes = set(graph.keys()) | set(h[0] for h in sum(graph.values(), [])) | set(heuristic.keys())
    g_score = {node: float('inf') for node in all_nodes}
    g_score[start] = 0


    h_start = heuristic.get(start, float('inf'))
    if h_start != float('inf'):
        heapq.heappush(open_set, (h_start, start))

    came_from = {}

    while open_set:
        current_f_score, current_node = heapq.heappop(open_set)

        if current_node == goal:
            return reconstruct_path(came_from, current_node), g_score[goal]

        for neighbor, cost in graph.get(current_node, []):

            tentative_g_score = g_score[current_node] + cost

            if tentative_g_score < g_score.get(neighbor, float('inf')):

                came_from[neighbor] = current_node
                g_score[neighbor] = tentative_g_score

                f_score = tentative_g_score + heuristic.get(neighbor, 0)

                heapq.heappush(open_set, (f_score, neighbor))

    return "No path found.", float('inf')

def reconstruct_path(came_from, current):
    """Reconstructs the path from the came_from map."""
    total_path = [current]
    while current in came_from:
        current = came_from[current]
        total_path.append(current)
    return total_path[::-1]
if __name__ == "__main__":

    graph = {
        'A': [('B', 1), ('C', 4)],
        'B': [('C', 2), ('D', 5)],
        'C': [('D', 2), ('E', 8)],
        'D': [('E', 3), ('Goal', 6)],
        'E': [('Goal', 1)],
        'Goal': []
    }

    heuristic = {
        'A': 8,
        'B': 6,
        'C': 5,
        'D': 3,
        'E': 1,
        'Goal': 0
    }

    START_NODE = 'A'
    GOAL_NODE = 'Goal'

    print("--- A* SEARCH ALGORITHM ---")
    print(f"Finding optimal path from {START_NODE} to {GOAL_NODE}...")

    path, cost = a_star_search(graph, heuristic, START_NODE, GOAL_NODE)

    if cost != float('inf'):
        print("\nOptimal Path Found:")
        print(" -> ".join(path))
        print(f"\nTotal Cost: {cost}")
    else:
        print("No path found.")

--- A* SEARCH ALGORITHM ---
Finding optimal path from A to Goal...

Optimal Path Found:
A -> B -> C -> D -> E -> Goal

Total Cost: 9


EXPERIMENT 5

In [3]:
import time

PLAYER_X = 'X'
PLAYER_O = 'O'
EMPTY = ' '

def check_winner(board):
    """Returns 'X', 'O', 'Draw', or None."""

    for i in range(3):
        if board[i][0] == board[i][1] == board[i][2] != EMPTY:
            return board[i][0]
        if board[0][i] == board[1][i] == board[2][i] != EMPTY:
            return board[0][i]

    if board[0][0] == board[1][1] == board[2][2] != EMPTY:
        return board[0][0]
    if board[0][2] == board[1][1] == board[2][0] != EMPTY:
        return board[0][2]

    if all(board[r][c] != EMPTY for r in range(3) for c in range(3)):
        return 'Draw'

    return None

def minimax_alpha_beta(board, depth, is_maximizing_player, alpha, beta):
    """
    Minimax function with Alpha-Beta Pruning.
    Returns the optimal score for the current state.
    """

    winner = check_winner(board)

    if winner:
        if winner == PLAYER_X:
            return 10 - depth
        elif winner == PLAYER_O:
            return depth - 10
        elif winner == 'Draw':
            return 0



    if is_maximizing_player:
        max_eval = float('-inf')

        for r in range(3):
            for c in range(3):
                if board[r][c] == EMPTY:
                    board[r][c] = PLAYER_X

                    eval_score = minimax_alpha_beta(board, depth + 1, False, alpha, beta)

                    board[r][c] = EMPTY

                    max_eval = max(max_eval, eval_score)
                    alpha = max(alpha, max_eval)
                    if beta <= alpha:
                        return max_eval

        return max_eval

    else:
        min_eval = float('inf')

        for r in range(3):
            for c in range(3):
                if board[r][c] == EMPTY:
                    board[r][c] = PLAYER_O

                    eval_score = minimax_alpha_beta(board, depth + 1, True, alpha, beta)

                    board[r][c] = EMPTY
                    min_eval = min(min_eval, eval_score)
                    beta = min(beta, min_eval)

                    if beta <= alpha:
                        return min_eval

        return min_eval

def find_best_move(board):
    """Determines the best move (row, col) for the 'X' player."""
    best_eval = float('-inf')
    best_move = None

    for r in range(3):
        for c in range(3):
            if board[r][c] == EMPTY:
                board[r][c] = PLAYER_X

                move_eval = minimax_alpha_beta(board, 0, False, float('-inf'), float('inf'))

                board[r][c] = EMPTY
                if move_eval > best_eval:
                    best_eval = move_eval
                    best_move = (r, c)

    return best_move, best_eval

if __name__ == "__main__":


    initial_board = [
        ['X', 'O', EMPTY],
        ['O', 'O', EMPTY],
        ['X', EMPTY, EMPTY]
    ]

    print("\n" + "=" * 50)
    print("  MINIMAX with ALPHA-BETA PRUNING (TIC-TAC-TOE)  ")
    print("=" * 50)

    def print_board(board):
        for row in board:
            print(f"| {' | '.join(row)} |")

    print("\nInitial Board State (X to move):")
    print_board(initial_board)

    start_time = time.time()
    best_move, best_eval = find_best_move(initial_board)
    end_time = time.time()

    print(f"\nAI (X) Best Move: Position ({best_move[0] + 1}, {best_move[1] + 1})")
    print(f"Optimal Evaluation Score: {best_eval}")
    print(f"Time taken for calculation: {end_time - start_time:.6f} seconds")


    if best_move:
        initial_board[best_move[0]][best_move[1]] = PLAYER_X
        print("\nResulting Board:")
        print_board(initial_board)


  MINIMAX with ALPHA-BETA PRUNING (TIC-TAC-TOE)  

Initial Board State (X to move):
| X | O |   |
| O | O |   |
| X |   |   |

AI (X) Best Move: Position (1, 3)
Optimal Evaluation Score: -9
Time taken for calculation: 0.000127 seconds

Resulting Board:
| X | O | X |
| O | O |   |
| X |   |   |


EXPERIMENT 6

In [2]:
import time
import copy

def print_grid(grid):
    """Helper function to print the Sudoku grid in a readable format."""
    print("-" * 25)
    for r in range(9):
        if r % 3 == 0 and r != 0:
            print("|" + "---+" * 8 + "---|") e

        row_str = "|"
        for c in range(9):
            if c % 3 == 0 and c != 0:
                row_str += " |"

            cell_value = str(grid[r][c]) if grid[r][c] != 0 else ' '
            row_str += f" {cell_value}"

        row_str += " |"
        print(row_str)
    print("-" * 25)

def find_empty_cell(grid):
    """Finds the next empty cell (represented by 0) to fill. (Simple variable ordering)"""
    for r in range(9):
        for c in range(9):
            if grid[r][c] == 0:
                return r, c
    return None

def is_safe(grid, row, col, num):
    """Checks if 'num' is a valid placement at grid[row][col] (Row, Column, and 3x3 Box Constraints)."""

    for c in range(9):
        if grid[row][c] == num:
            return False

    for r in range(9):
        if grid[r][col] == num:
            return False

    box_start_row = 3 * (row // 3)
    box_start_col = 3 * (col // 3)

    for r in range(box_start_row, box_start_row + 3):
        for c in range(box_start_col, box_start_col + 3):
            if grid[r][c] == num:
                return False

    return True

def solve_sudoku(grid):
    """
    Main backtracking function to solve the Sudoku CSP.
    Modifies the grid in-place.
    """

    empty_cell = find_empty_cell(grid)

    if not empty_cell:
        return True

    row, col = empty_cell

    for num in range(1, 10):

        if is_safe(grid, row, col, num):

            grid[row][col] = num

            if solve_sudoku(grid):
                return True


            grid[row][col] = 0


    return False


if __name__ == "__main__":


    puzzle_unsolved = [
        [5, 3, 0, 0, 7, 0, 0, 0, 0],
        [6, 0, 0, 1, 9, 5, 0, 0, 0],
        [0, 9, 8, 0, 0, 0, 0, 6, 0],

        [8, 0, 0, 0, 6, 0, 0, 0, 3],
        [4, 0, 0, 8, 0, 3, 0, 0, 1],
        [7, 0, 0, 0, 2, 0, 0, 0, 6],

        [0, 6, 0, 0, 0, 0, 2, 8, 0],
        [0, 0, 0, 4, 1, 9, 0, 0, 5],
        [0, 0, 0, 0, 8, 0, 0, 7, 9]
    ]

    puzzle_to_solve = copy.deepcopy(puzzle_unsolved)

    print("\n" + "=" * 30)
    print("   SUDOKU CSP SOLVER (Backtracking)  ")
    print("=" * 30)

    print("\nUnsolved Puzzle:")
    print_grid(puzzle_to_solve)

    start_time = time.time()

    if solve_sudoku(puzzle_to_solve):
        end_time = time.time()
        print("\n" + "-" * 25)
        print("Solution Found:")
        print_grid(puzzle_to_solve)
        print(f"Time taken to solve: {end_time - start_time:.4f} seconds")
    else:
        print("\nNo solution exists for this puzzle.")

SyntaxError: invalid syntax (ipython-input-2716506623.py, line 9)

EXPERIMENT 7

In [1]:
def parse_literal(literal):
    """Returns the variable (absolute value) and its sign (True for positive)."""
    return abs(literal), literal > 0

def unit_propagation(clauses, assignment):
    """
    Repeatedly applies the unit clause rule: if a clause has only one
    unassigned literal, assign that literal the value required to satisfy the clause.

    Returns: (new_clauses, new_assignment, conflict_detected)
    """
    new_assignment = assignment.copy()

    while True:
        unit_clause = None


        for clause in clauses:
            unassigned_literals = [
                literal for literal in clause
                if abs(literal) not in new_assignment
            ]

            if not unassigned_literals:
                return clauses, new_assignment, True

            if len(unassigned_literals) == 1:
                unit_clause = unassigned_literals[0]
                break

        if unit_clause is None:
            break

        var, sign = parse_literal(unit_clause)
        new_assignment[var] = sign

        new_clauses = []
        for clause in clauses:
            is_satisfied = False
            new_clause = []
            for literal in clause:
                var_l, sign_l = parse_literal(literal)

                if var_l in new_assignment:
                    if new_assignment[var_l] == sign_l:
                        is_satisfied = True
                        break

                else:
                    new_clause.append(literal)

            if not is_satisfied:
                if not new_clause:
                    return clauses, new_assignment, True
                new_clauses.append(new_clause)

        clauses = new_clauses

    return clauses, new_assignment, False


def dpll(clauses, assignment):
    """
    The main DPLL recursive function.
    """


    clauses, assignment, conflict = unit_propagation(clauses, assignment)
    if conflict:
        return False, None


    if not clauses:
        return True, assignment


    unassigned_vars = set()
    for clause in clauses:
        for literal in clause:
            var = abs(literal)
            if var not in assignment:
                unassigned_vars.add(var)

    if not unassigned_vars:
        return False, None

    p = next(iter(unassigned_vars))


    new_clauses_true = [
        [l for l in clause if l != -p]
        for clause in clauses if p not in clause
    ]
    new_assignment_true = assignment.copy()
    new_assignment_true[p] = True

    is_sat, model = dpll(new_clauses_true, new_assignment_true)
    if is_sat:
        return True, model


    new_clauses_false = [
        [l for l in clause if l != p]
        for clause in clauses if -p not in clause
    ]
    new_assignment_false = assignment.copy()
    new_assignment_false[p] = False

    is_sat, model = dpll(new_clauses_false, new_assignment_false)
    if is_sat:
        return True, model

    return False, None


def check_satisfiability(cnf_formula):
    """Runs the DPLL solver and prints results."""

    if any(not clause for clause in cnf_formula):
        print("Formula is UNSATISFIABLE (contains an empty clause).")
        return False, None

    is_sat, model = dpll(cnf_formula, {})

    if is_sat:
        final_model = {f'P{k}': v for k, v in model.items()}
        print("Formula is SATISFIABLE.")
        print(f"Model: {final_model}")
    else:
        print("Formula is UNSATISFIABLE.")

    return is_sat, model



print("--- TEST CASE: UNSATISFIABLE ---")
formula_unsat = [
    [1, 2],
    [-1, 2],
    [-2]
]
check_satisfiability(formula_unsat)

print("\n" + "="*50 + "\n")


print("--- TEST CASE: SATISFIABLE ---")
formula_sat = [
    [1],
    [2, -3]
]
check_satisfiability(formula_sat)

--- TEST CASE: UNSATISFIABLE ---
Formula is UNSATISFIABLE.


--- TEST CASE: SATISFIABLE ---
Formula is SATISFIABLE.
Model: {'P1': True, 'P2': True}


(True, {1: True, 2: True})

EXPERIMENT 8

In [6]:
import collections

def forward_chaining(knowledge_base, initial_facts, query):
    """
    Implements the Forward Chaining inference algorithm.

    Args:
        knowledge_base (list): A list of rules. Each rule is a dict
                               with 'premise' (list of facts) and
                               'conclusion' (a single fact).
        initial_facts (set): The set of facts known to be true initially.
        query (str): The proposition we are trying to prove.

    Returns:
        bool: True if the query is provable, False otherwise.
    """


    inferred = set(initial_facts)


    count = {}
    for i, rule in enumerate(knowledge_base):
        count[i] = len(rule['premise'])


    agenda = collections.deque(initial_facts)

    if query in inferred:
        print(f"Query '{query}' is already an initial fact.")
        return True

    print(f"Starting facts: {initial_facts}")
    print(f"Goal: Prove '{query}'")
    print("-" * 30)


    while agenda:
        p = agenda.popleft()

        print(f"-> Processing newly derived fact: {p}")

        for i, rule in enumerate(knowledge_base):
            if p in rule['premise']:
                count[i] -= 1

                if count[i] == 0:
                    q = rule['conclusion']

                    rule_str = " ^ ".join(rule['premise']) + f" => {q}"
                    print(f"  Rule satisfied: ({rule_str})")

                    if q not in inferred:
                        inferred.add(q)
                        agenda.append(q)
                        print(f"  New fact derived and added to agenda: {q}")

                        if q == query:
                            print("-" * 30)
                            print(f"[SUCCESS] Query '{query}' proven true.")
                            return True

    print("-" * 30)
    print(f"[FAIL] Agenda is empty. Query '{query}' not proven.")
    print(f"All inferred facts: {sorted(list(inferred))}")
    return False


knowledge_base_example = [
    {'premise': ['A', 'B'], 'conclusion': 'D'},
    {'premise': ['C', 'D'], 'conclusion': 'E'},
    {'premise': ['F'], 'conclusion': 'G'},
    {'premise': ['G', 'H'], 'conclusion': 'I'},
    {'premise': ['E'], 'conclusion': 'J'}
]

print("--- TEST CASE 1: PROVABLE QUERY (E) ---")
facts_1 = {'A', 'B', 'C'}
query_1 = 'E'
forward_chaining(knowledge_base_example, facts_1, query_1)

print("\n" + "="*50 + "\n")

print("--- TEST CASE 2: UNPROVABLE QUERY (I) ---")
facts_2 = {'A', 'B', 'C', 'F'}
query_2 = 'I'
forward_chaining(knowledge_base_example, facts_2, query_2)

--- TEST CASE 1: PROVABLE QUERY (E) ---
Starting facts: {'A', 'C', 'B'}
Goal: Prove 'E'
------------------------------
-> Processing newly derived fact: A
-> Processing newly derived fact: C
-> Processing newly derived fact: B
  Rule satisfied: (A ^ B => D)
  New fact derived and added to agenda: D
-> Processing newly derived fact: D
  Rule satisfied: (C ^ D => E)
  New fact derived and added to agenda: E
------------------------------
[SUCCESS] Query 'E' proven true.


--- TEST CASE 2: UNPROVABLE QUERY (I) ---
Starting facts: {'F', 'A', 'C', 'B'}
Goal: Prove 'I'
------------------------------
-> Processing newly derived fact: F
  Rule satisfied: (F => G)
  New fact derived and added to agenda: G
-> Processing newly derived fact: A
-> Processing newly derived fact: C
-> Processing newly derived fact: B
  Rule satisfied: (A ^ B => D)
  New fact derived and added to agenda: D
-> Processing newly derived fact: G
-> Processing newly derived fact: D
  Rule satisfied: (C ^ D => E)
  New fact

False

EXPERIMENT 9


In [7]:
import collections


_proven_facts = set()
_knowledge_base = []


def is_provable_backward(query):
    """
    The core recursive function for the Backward Chaining Algorithm.

    Args:
        query (str): The current sub-goal (fact) we are trying to prove.

    Returns:
        bool: True if the query can be proven, False otherwise.
    """
    global _proven_facts
    global _knowledge_base

    print(f"-> Goal: Prove '{query}'")

    if query in _proven_facts:
        print(f"  [KNOWN] Fact '{query}' is already proven.")
        return True

    relevant_rules = [rule for rule in _knowledge_base if rule['conclusion'] == query]

    if not relevant_rules:
        print(f"  [FAIL] No rule concludes '{query}'. Cannot prove.")
        return False

    for rule in relevant_rules:
        all_premises_satisfied = True

        rule_str = " ^ ".join(rule['premise']) + f" => {query}"
        print(f"  Trying rule: ({rule_str})")

        for premise in rule['premise']:
            if not is_provable_backward(premise):
                all_premises_satisfied = False
                break

        if all_premises_satisfied:
            _proven_facts.add(query)
            print(f"  [SUCCESS] All premises satisfied. '{query}' is PROVEN via rule: ({rule_str})")
            return True

    return False


def backward_chaining_solver(knowledge_base, initial_facts, query):
    """
    Public function to initialize and start the Backward Chaining process.
    """
    global _proven_facts
    global _knowledge_base

    _proven_facts.clear()

    _proven_facts.update(initial_facts)
    _knowledge_base = knowledge_base

    print("--- Starting Backward Chaining Solver ---")
    print(f"Initial Facts (Base Cases): {initial_facts}")
    print(f"Target Query: {query}")
    print("-" * 40)

    result = is_provable_backward(query)

    print("-" * 40)
    if result:
        print(f"[FINAL RESULT] Query '{query}' is PROVABLE.")
    else:
        print(f"[FINAL RESULT] Query '{query}' is UNPROVABLE.")

    return result


knowledge_base_example = [
    {'premise': ['A', 'B'], 'conclusion': 'D'},
    {'premise': ['C', 'D'], 'conclusion': 'E'},
    {'premise': ['F'], 'conclusion': 'G'},
    {'premise': ['G', 'H'], 'conclusion': 'I'},
    {'premise': ['E'], 'conclusion': 'J'}
]

print("--- TEST CASE 1: PROVABLE QUERY (E) ---")
facts_1 = {'A', 'B', 'C'}
query_1 = 'E'
backward_chaining_solver(knowledge_base_example, facts_1, query_1)

print("\n" + "="*50 + "\n")

print("--- TEST CASE 2: UNPROVABLE QUERY (I) ---")
facts_2 = {'A', 'B', 'C', 'F'}
query_2 = 'I'
backward_chaining_solver(knowledge_base_example, facts_2, query_2)

--- TEST CASE 1: PROVABLE QUERY (E) ---
--- Starting Backward Chaining Solver ---
Initial Facts (Base Cases): {'A', 'C', 'B'}
Target Query: E
----------------------------------------
-> Goal: Prove 'E'
  Trying rule: (C ^ D => E)
-> Goal: Prove 'C'
  [KNOWN] Fact 'C' is already proven.
-> Goal: Prove 'D'
  Trying rule: (A ^ B => D)
-> Goal: Prove 'A'
  [KNOWN] Fact 'A' is already proven.
-> Goal: Prove 'B'
  [KNOWN] Fact 'B' is already proven.
  [SUCCESS] All premises satisfied. 'D' is PROVEN via rule: (A ^ B => D)
  [SUCCESS] All premises satisfied. 'E' is PROVEN via rule: (C ^ D => E)
----------------------------------------
[FINAL RESULT] Query 'E' is PROVABLE.


--- TEST CASE 2: UNPROVABLE QUERY (I) ---
--- Starting Backward Chaining Solver ---
Initial Facts (Base Cases): {'F', 'A', 'C', 'B'}
Target Query: I
----------------------------------------
-> Goal: Prove 'I'
  Trying rule: (G ^ H => I)
-> Goal: Prove 'G'
  Trying rule: (F => G)
-> Goal: Prove 'F'
  [KNOWN] Fact 'F' is alre

False

EXPERIMENT 10

In [8]:
import collections
import math

class NaiveBayesClassifier:
    """
    A simple implementation of the Categorical Naive Bayes classifier.
    It calculates P(C_k) and P(x_i | C_k) using frequency counts and
    Laplace smoothing.
    """

    def __init__(self):

        self.class_priors = {}
        self.likelihoods = collections.defaultdict(lambda: collections.defaultdict(lambda: collections.defaultdict(float)))
        self.classes = set()
        self.feature_names = []
        self.class_counts = collections.defaultdict(int)
        self.feature_counts = collections.defaultdict(lambda: collections.defaultdict(lambda: collections.defaultdict(int)))

    def train(self, X, Y):
        """
        Trains the classifier by calculating priors and likelihoods.

        Args:
            X (list of dict): Training features.
            Y (list): Training labels (class names).
        """
        if not X or len(X) != len(Y):
            raise ValueError("Input data (X) and labels (Y) must be non-empty and have the same length.")

        N = len(X)
        self.classes = set(Y)
        self.feature_names = list(X[0].keys())

        for features, label in zip(X, Y):
            self.class_counts[label] += 1
            for feature_name, value in features.items():
                self.feature_counts[label][feature_name][value] += 1

        for cls in self.classes:
            self.class_priors[cls] = self.class_counts[cls] / N

        alpha = 1

        for cls in self.classes:
            num_samples_in_class = self.class_counts[cls]

            for feature_name in self.feature_names:
                all_feature_values = set(d.get(feature_name) for d in X)
                V = len(all_feature_values)

                for value in all_feature_values:
                    count = self.feature_counts[cls][feature_name][value]
                    self.likelihoods[cls][feature_name][value] = (count + alpha) / (num_samples_in_class + alpha * V)

    def predict(self, features):
        """
        Predicts the class label for a single new example. Uses log-probabilities
        to prevent underflow and simplify multiplication.

        Args:
            features (dict): The input example.

        Returns:
            str: The predicted class label.
        """
        best_class = None
        max_log_probability = -float('inf')

        print("\n--- Prediction Log ---")

        for cls, prior in self.class_priors.items():
            log_probability = math.log(prior)

            print(f"Checking Class '{cls}': Log Prior = {log_probability:.4f}")

            for feature_name, value in features.items():

                likelihood = self.likelihoods[cls][feature_name].get(value)

                if likelihood is None:
                    likelihood = 1e-9

                log_probability += math.log(likelihood)
                print(f"  Feature {feature_name}='{value}': Log Likelihood = {math.log(likelihood):.4f}")

            print(f"Total Log Probability for '{cls}': {log_probability:.4f}")

            if log_probability > max_log_probability:
                max_log_probability = log_probability
                best_class = cls

        print("-" * 20)
        return best_class

def run_naive_bayes_example():
    """Sets up the training data and runs the classification example."""

    data = [
        {'Outlook': 'Sunny', 'Temp': 'Hot', 'Humidity': 'High', 'Wind': 'Weak'},
        {'Outlook': 'Sunny', 'Temp': 'Hot', 'Humidity': 'High', 'Wind': 'Strong'},
        {'Outlook': 'Overcast', 'Temp': 'Hot', 'Humidity': 'High', 'Wind': 'Weak'},
        {'Outlook': 'Rain', 'Temp': 'Mild', 'Humidity': 'High', 'Wind': 'Weak'},
        {'Outlook': 'Rain', 'Temp': 'Cool', 'Humidity': 'Normal', 'Wind': 'Weak'},
        {'Outlook': 'Rain', 'Temp': 'Cool', 'Humidity': 'Normal', 'Wind': 'Strong'},
        {'Outlook': 'Overcast', 'Temp': 'Cool', 'Humidity': 'Normal', 'Wind': 'Strong'},
        {'Outlook': 'Sunny', 'Temp': 'Mild', 'Humidity': 'High', 'Wind': 'Weak'},
        {'Outlook': 'Sunny', 'Temp': 'Cool', 'Humidity': 'Normal', 'Wind': 'Weak'},
        {'Outlook': 'Rain', 'Temp': 'Mild', 'Humidity': 'Normal', 'Wind': 'Weak'},
        {'Outlook': 'Sunny', 'Temp': 'Mild', 'Humidity': 'Normal', 'Wind': 'Strong'},
        {'Outlook': 'Overcast', 'Temp': 'Mild', 'Humidity': 'High', 'Wind': 'Strong'},
        {'Outlook': 'Overcast', 'Temp': 'Hot', 'Humidity': 'Normal', 'Wind': 'Weak'},
        {'Outlook': 'Rain', 'Temp': 'Mild', 'Humidity': 'High', 'Wind': 'Strong'},
    ]

    labels = [
        'No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No'
    ]

    print("--- Training Naive Bayes Classifier ---")

    nb = NaiveBayesClassifier()
    nb.train(data, labels)

    print("\nTraining Complete.")
    print(f"Class Priors (P(C)): {nb.class_priors}")

    new_example = {
        'Outlook': 'Sunny',
        'Temp': 'Cool',
        'Humidity': 'High',
        'Wind': 'Strong'
    }

    print("\n" + "="*50)
    print("NEW EXAMPLE TO PREDICT:")
    print(new_example)

    prediction = nb.predict(new_example)

    print("\n" + "="*50)
    print(f"The predicted decision is: **{prediction}**")
    print("This means the model predicts 'No' for playing, as the combination of probabilities favors the 'No' class.")

if __name__ == '__main__':
    run_naive_bayes_example()

--- Training Naive Bayes Classifier ---

Training Complete.
Class Priors (P(C)): {'Yes': 0.6428571428571429, 'No': 0.35714285714285715}

NEW EXAMPLE TO PREDICT:
{'Outlook': 'Sunny', 'Temp': 'Cool', 'Humidity': 'High', 'Wind': 'Strong'}

--- Prediction Log ---
Checking Class 'Yes': Log Prior = -0.4418
  Feature Outlook='Sunny': Log Likelihood = -1.3863
  Feature Temp='Cool': Log Likelihood = -1.0986
  Feature Humidity='High': Log Likelihood = -1.0116
  Feature Wind='Strong': Log Likelihood = -1.0116
Total Log Probability for 'Yes': -4.9499
Checking Class 'No': Log Prior = -1.0296
  Feature Outlook='Sunny': Log Likelihood = -0.6931
  Feature Temp='Cool': Log Likelihood = -1.3863
  Feature Humidity='High': Log Likelihood = -0.3365
  Feature Wind='Strong': Log Likelihood = -0.5596
Total Log Probability for 'No': -4.0051
--------------------

The predicted decision is: **No**
This means the model predicts 'No' for playing, as the combination of probabilities favors the 'No' class.
